# Web Streaming

This notebook will stream video from the Raspberry Pi camera to a webpage (http://orcspi.local:8000) and you can watch your spectrometer **life**. The code is from the <a href="http://picamera.readthedocs.io/en/latest/recipes2.html#web-streaming">official PiCamera package</a>.

**Note:** You can only run **one** notebook using the camera at a time - so either this one to stream or your spectrometer notebook to collect spectra - at least for now 😎

### Server code

This code is the server - just run :)

In [ ]:
import io
import picamera
import logging
import socketserver
from threading import Condition
from http import server

In [ ]:
from IPython.display import HTML, IFrame

In [ ]:
PAGE="""\
<html>
<head>
<title>Raspberry Pi - Spectrometer Camera</title>
</head>
<body>
<center><h1>Raspberry Pi - Spectrometer Camera</h1></center>
<center><img src="stream.mjpg" width="640" height="480"></center>
</body>
</html>
"""

class StreamingOutput(object):
    def __init__(self):
        self.frame = None
        self.buffer = io.BytesIO()
        self.condition = Condition()

    def write(self, buf):
        if buf.startswith(b'\xff\xd8'):
            # New frame, copy the existing buffer's content and notify all
            # clients it's available
            self.buffer.truncate()
            with self.condition:
                self.frame = self.buffer.getvalue()
                self.condition.notify_all()
            self.buffer.seek(0)
        return self.buffer.write(buf)

class StreamingHandler(server.BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.send_response(301)
            self.send_header('Location', '/index.html')
            self.end_headers()
        elif self.path == '/index.html':
            content = PAGE.encode('utf-8')
            self.send_response(200)
            self.send_header('Content-Type', 'text/html')
            self.send_header('Content-Length', len(content))
            self.end_headers()
            self.wfile.write(content)
        elif self.path == '/stream.mjpg':
            self.send_response(200)
            self.send_header('Age', 0)
            self.send_header('Cache-Control', 'no-cache, private')
            self.send_header('Pragma', 'no-cache')
            self.send_header('Content-Type', 'multipart/x-mixed-replace; boundary=FRAME')
            self.end_headers()
            try:
                while True:
                    with output.condition:
                        output.condition.wait()
                        frame = output.frame
                    self.wfile.write(b'--FRAME\r\n')
                    self.send_header('Content-Type', 'image/jpeg')
                    self.send_header('Content-Length', len(frame))
                    self.end_headers()
                    self.wfile.write(frame)
                    self.wfile.write(b'\r\n')
            except Exception as e:
                logging.warning(
                    'Removed streaming client %s: %s',
                    self.client_address, str(e))
        else:
            self.send_error(404)
            self.end_headers()

class StreamingServer(socketserver.ThreadingMixIn, server.HTTPServer):
    allow_reuse_address = True
    daemon_threads = True
    

### Running server

And here we actually run the server. While this cell is running, you can access  http://orcspi.local:8000.

In [ ]:
with picamera.PiCamera(resolution='640x480', framerate=24) as camera, StreamingServer(('', 8000), StreamingHandler) as server:
    
    try:
        # This is to output the video in IFrame
        display(IFrame('http://orcspi.local:8000', width=800, height=600))

        output = StreamingOutput()
        camera.rotation = 270
        camera.start_recording(output, format='mjpeg')
        server.serve_forever()
    except KeyboardInterrupt:
        print('Interrupted')    
    finally:
        camera.stop_recording()